In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Echarts Jupyter lab support
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [ ]:
# Echarts imports
from pyecharts import options as opts
from pyecharts.globals import  ThemeType
from pyecharts.charts import Line, Grid

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt

In [ ]:
from common.log_helper import LogHelper
from backtest.scale_backtest_args import ScaleBacktestArgGen
from backtest.scale_backtest import ScaleBacktest
from modelservice.alpha_consts import SignalDirection
from notebooks.echart_bt_plots import *
from notebooks.echart_df_plot import *
from datetime import datetime
from datetime import timedelta
import requests
import json
import logging
import sys
import pandas as pd

In [ ]:
#LogHelper.configure_logging(verbose=False)
pd.set_option("display.precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
def colorize(val):
    if val == 'B':
        color = 'green'
    elif val == 'S':
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color

In [ ]:
BtArgs = {
    'common': {
        'start_time': '09:15:01',
        'end_time' : '15:29:00',
    },
    'alpha': {
        'sigmult' : 0.5,
        'smoother' : {
            'enabled' : False,
        }
    },
    'execution': {
        'exec_instr_type' : 'NK_F',        
        'price_type' : 'LTP',
        'stop_loss': 0,
        'max_loss' : 0,
    }
}

In [ ]:
dump_base_dir = "/home/qoptrprod/data/dumps/"
trading_date = "20240319"
underlying = "MIDCPNIFTY_I"

In [ ]:
ul = underlying.split('_')[0]
url = f'http://someurl.com/api/chain_app/runcmpbt/?ul={ul}&sdate={trading_date}&edate={trading_date}'
#print(url)
response = requests.get(url=url)

In [ ]:
qoptr_dict_list = json.loads(response.text)
qoptr_day_result = qoptr_dict_list[0]['day_results'][0]
qoptr_signals = qoptr_day_result['signals']
print("QOPTR Results")
pd.DataFrame([qoptr_day_result['summary']])

In [ ]:
arg_obj = ScaleBacktestArgGen.resolve_and_gen_args(
    dump_base_dir=dump_base_dir,
    underlying=underlying,
    trading_date=trading_date,
    bt_args=BtArgs
)
bt = ScaleBacktest(arg_obj)
result = bt.run_bt_with_qoptr_signal(qoptr_signals, shift=1)
trade_df = pd.DataFrame(result['trademarkers'])

In [ ]:
def buy_or_sell(v):
    if v == 'BUY':
        return 'color : lightgreen'
    elif v == 'SELL':
        return 'color : red'
    elif v == 'OPEN':        
        return 'background-color : darkorange'   
    elif v == None:
        return ''
    return ''

In [ ]:
sum_df = pd.DataFrame([result['summary']])
sum_df

In [ ]:
#trade_df.style.map(lambda x: buy_or_sell(x)).format({col: "{:.1f}" for col in ['price','profit','cost','to','sigval', 'cpx','ppx','ulpx', 'atm']})

### Signal Run To For Detailed Plots

In [ ]:
sig_args_obj = arg_obj
bt = ScaleBacktest(sig_args_obj)
result = bt.run_signal()

In [ ]:
# create signal_df with additional time stamp column for comparisons
signal_df = pd.DataFrame(result)
signal_df['ts'] = pd.to_datetime(signal_df['ulexts'])

In [ ]:
trade_plot = plot_trade_signals(signal_df, trade_df, sum_df, show_pnl_markers=True, show_legends=True)

In [ ]:
trade_plot.load_javascript()

In [ ]:
trade_plot.render_notebook()

### QSignal vs Scale Signal

In [ ]:
qs_df = pd.DataFrame(qoptr_signals)
qs_df['ts'] = pd.to_datetime(qs_df['ulexts'])

In [ ]:
display(signal_df.head(1))
display(qs_df.head(1))

In [ ]:
shift_min = 0

In [ ]:
merge_list = []
color_none = 'rgba(100, 100, 100, 0.5)'

color_red_1 = 'rgba(230,50,100, 0.5)' 
color_green_1 = 'rgba(50,200,100,0.5)'

color_red_2 = 'rgba(230,50,100,0.9)' 
color_green_2 = 'rgba(50,200,100,0.9)'

q_mp_val = []
s_mp_val = []
q_mp_ul = []
s_mp_ul = []

for _, qrow in qs_df.iterrows():
    scale = signal_df.loc[signal_df.ts == qrow.ts - timedelta(minutes=shift_min) ]
    if not scale.empty:
        str_time = scale.ulexts.values[0].split(' ')[1]
        merge_list.append({'ulexts' : str_time, 
                           'qoptr_ulpx' : qrow.ulpx, 
                           'scale_ulpx' : scale.ulpx.values[0],

                           'qoptr_atm' : qrow.atm, 
                           'scale_atm' : scale.atm.values[0],
                           
                           'qoptr_cpx' : qrow.cpx, 
                           'scale_cpx' : scale.cpx.values[0],

                           'qoptr_ppx' : qrow.ppx, 
                           'scale_ppx' : scale.ppx.values[0],

                           'qoptr_val' : qrow.val, 
                           'scale_val' : scale.val.values[0],
                          }
                         )
        if qrow.dir == SignalDirection.BUY or qrow.dir == SignalDirection.SELL:
            color = color_none
            if qrow.dir == SignalDirection.BUY:
                color = color_green_1
            elif qrow.dir == SignalDirection.SELL:
                color = color_red_1
            q_mp_val.append(opts.MarkPointItem(coord=[str_time, qrow.val], 
                    symbol='circle',               
                    symbol_size=[12, 12],
                    itemstyle_opts=opts.ItemStyleOpts(color=color),
                    ))
            q_mp_ul.append(opts.MarkPointItem(coord=[str_time, qrow.ulpx], 
                    symbol='circle',               
                    symbol_size=[12, 12],
                    itemstyle_opts=opts.ItemStyleOpts(color=color),
                    ))
        
        scale_dir = scale.dir.values[0]    
        if scale_dir == SignalDirection.BUY or scale_dir == SignalDirection.SELL:
            color = color_none
            if scale_dir == SignalDirection.BUY:
                color = color_green_2
            elif scale_dir == SignalDirection.SELL:
                color = color_red_2
            s_mp_val.append(opts.MarkPointItem(coord=[str_time, scale.val.values[0]], 
                    symbol='circle',               
                    symbol_size=[7, 7],
                    itemstyle_opts=opts.ItemStyleOpts(color=color),
                    ))
            s_mp_ul.append(opts.MarkPointItem(coord=[str_time, scale.ulpx.values[0]], 
                    symbol='circle',               
                    symbol_size=[7, 7],
                    itemstyle_opts=opts.ItemStyleOpts(color=color),
                    ))
            
merge_df = pd.DataFrame(merge_list)      
merge_df.head(1)

In [ ]:
merge_ulpx_plot = plot_df_simple(merge_df, title="QOPTR vs Scale Underlying", x_col='ulexts', y_cols=['qoptr_ulpx', 'scale_ulpx'],
                                mark_points={
                                    'qoptr_ulpx' : q_mp_ul,
                                    'scale_ulpx' : s_mp_ul,
                                }
                                    )
merge_atm_plot = plot_df_simple(merge_df, title="QOPTR vs Scale ATM", x_col='ulexts', y_cols=['qoptr_atm', 'scale_atm'])
merge_val_plot = plot_df_simple(merge_df, title="QOPTR vs Scale Signal Values", x_col='ulexts', y_cols=['qoptr_val', 'scale_val'],
                                mark_points={
                                    'qoptr_val' : q_mp_val,
                                    'scale_val' : s_mp_val
                                    }
                               )
merge_cpx_plot = plot_df_simple(merge_df, title="QOPTR vs Scale Call Prices", x_col='ulexts', y_cols=['qoptr_cpx', 'scale_cpx'])
merge_ppx_plot = plot_df_simple(merge_df, title="QOPTR vs Scale Put Prices", x_col='ulexts', y_cols=['qoptr_ppx', 'scale_ppx'])

In [ ]:
merge_ulpx_plot.load_javascript()
merge_atm_plot.load_javascript()
merge_val_plot.load_javascript()
merge_cpx_plot.load_javascript()
merge_ppx_plot.load_javascript()

In [ ]:
merge_ulpx_plot.render_notebook()

In [ ]:
merge_atm_plot.render_notebook()

In [ ]:
merge_val_plot.render_notebook()

In [ ]:
merge_cpx_plot.render_notebook()

In [ ]:
merge_ppx_plot.render_notebook()

In [ ]:
merge_val_plot.set_global_opts(
    axispointer_opts=opts.AxisPointerOpts(
            is_show=True, link=[{"xAxisIndex": "all"}]
        ),
    datazoom_opts=[
            opts.DataZoomOpts(
                is_show=True,
                is_realtime=True,
                start_value=30,
                end_value=70,
                xaxis_index=[0, 1],
            )
        ],
    legend_opts=opts.LegendOpts(pos_left="left"),
)

In [ ]:
merge_atm_plot.set_global_opts(
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True, link=[{"xAxisIndex": "all"}]
        ),
        xaxis_opts=opts.AxisOpts(
            grid_index=1,
            position="top",
        ),
        datazoom_opts=[
            opts.DataZoomOpts(
                is_realtime=True,
                type_="inside",
                xaxis_index=[0, 1],
            )
        ],
        legend_opts=opts.LegendOpts(pos_left="7%"),
    )

In [ ]:
grid = (Grid(init_opts=opts.InitOpts(theme=ThemeType.DARK, width='1200px', height='500px'))
        .add(merge_val_plot, grid_opts=opts.GridOpts(pos_left=50, pos_right=50, height="35%"))
        .add(merge_atm_plot, grid_opts=opts.GridOpts(pos_left=50, pos_right=50, pos_top="55%", height="35%"))
       )
grid.load_javascript()

In [ ]:
grid.render_notebook()